<h1> Setting up HF and packages </h1>

In [ ]:
from huggingface_hub import notebook_login
notebook_login()To run This This ThisThis fdjsafjdisoathioa;fdoasifdj;aosidjf;aosidjf;aosidjf;aosdifj

/Users/kunalbhandarkar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
!huggingface-cli whoami

/Users/kunalbhandarkar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
kunalb123


In [3]:
from llama_cpp import Llama
import time
import torch
from transformers import AutoTokenizer
from huggingface_hub import hf_hub_download
import psutil
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm
import ast
import re

<h1> CPU Model </h1>

In [4]:
class CPUModel:
    def __init__(self, args):
        """Initialize the model."""
        model_path = hf_hub_download(repo_id=args["repo_id"], filename=args["filename"])
        self.model = Llama(model_path=model_path, verbose=False)
        if args['repo_id'] == "bartowski/DeepSeek-R1-Distill-Qwen-1.5B-GGUF":
            self.tokenizer = AutoTokenizer.from_pretrained('./tokenizers/toknizer_1.5/')
        elif args['repo_id'] == "bartowski/DeepSeek-R1-Distill-Qwen-7B-GGUF":
            self.tokenizer = AutoTokenizer.from_pretrained('./tokenizers/tokenizer_7/')
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(args["tokenizer_path"], use_auth_token=args.get("auth"))

    def get_response(self, args):
        """Generate response from the model."""
        return self.model(args["prompt"], max_tokens=args.get("max_tokens", 100))

    def get_text_response(self, args):
        """Extract text response from model output."""
        output = self.get_response(args)
        return output['choices'][0]['text']

    def evaluate(self, args):
        """Measure number of tokens, latency, throughput, and RAM usage."""
        start_time = time.time()
        
        output_text = self.get_text_response(args)
        latency = time.time() - start_time

        # Get initial RAM usage
        process = psutil.Process()
        
        # Get peak RAM usage after execution
        max_ram = process.memory_info().rss / (1024 ** 3)  # Convert to GB
        
        # Tokenize the output for accurate measurement
        tokenized_output = self.tokenizer(output_text)
        tokens_generated = len(tokenized_output['input_ids'])
        
        
        # Throughput calculation (tokens per second)
        throughput = tokens_generated / latency
        
        return {
            "latency": latency,
            "throughput": throughput,
            "max_ram": max_ram,
            "tokens_generated": tokens_generated,
            "model_output_str": output_text, 
        }

<h1> Models List</h1>

In [30]:
batch_models = [
    {
        "repo_id": "Qwen/Qwen2-0.5B-Instruct-GGUF",
        "filename": "qwen2-0_5b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "Qwen/Qwen2-1.5B-Instruct-GGUF",
        "filename": "qwen2-1_5b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "Qwen/Qwen2-7B-Instruct-GGUF",
        "filename": "qwen2-7b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-3B-Instruct-GGUF",
        "filename": "Llama-3.2-3B-Instruct.Q4_K_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
]

model_info = [ 
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-1B-Instruct-GGUF",
        "filename": "Llama-3.2-1B-Instruct.Q4_K_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "bartowski/Llama-3.2-3B-Instruct-GGUF",
        "filename": "Llama-3.2-3B-Instruct-Q4_K_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B"
    },
    {
        "repo_id": "bartowski/Meta-Llama-3.1-8B-Instruct-GGUF",
        "filename": "Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B"
    },
    {
        "repo_id": "Qwen/Qwen2-1.5B-Instruct-GGUF",
        "filename": "qwen2-1_5b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "Qwen/Qwen2-7B-Instruct-GGUF",
        "filename": "qwen2-7b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "Qwen/Qwen2-0.5B-Instruct-GGUF",
        "filename": "qwen2-0_5b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "bartowski/DeepSeek-R1-Distill-Qwen-1.5B-GGUF",
        "filename": "DeepSeek-R1-Distill-Qwen-1.5B-Q6_K.gguf",
        "tokenizer_path": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    },
    {
        "repo_id": "bartowski/DeepSeek-R1-Distill-Qwen-1.5B-GGUF",
        "filename": "DeepSeek-R1-Distill-Qwen-1.5B-f32.gguf",
        "tokenizer_path": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    },
    {
        "repo_id": "bartowski/DeepSeek-R1-Distill-Qwen-1.5B-GGUF",
        "filename": "DeepSeek-R1-Distill-Qwen-1.5B-Q8_0.gguf",
        "tokenizer_path": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    },
    {
        "repo_id": "bartowski/DeepSeek-R1-Distill-Qwen-1.5B-GGUF",
        "filename": "DeepSeek-R1-Distill-Qwen-1.5B-Q4_K_M.gguf",
        "tokenizer_path": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    },
    {
        "repo_id": "bartowski/DeepSeek-R1-Distill-Qwen-7B-GGUF",
        "filename": "DeepSeek-R1-Distill-Qwen-7B-Q6_K.gguf",
        "tokenizer_path": "deepseek-ai/DeepSeek-R1-7B"
    },
    {
        "repo_id": "bartowski/DeepSeek-R1-Distill-Qwen-7B-GGUF",
        "filename": "DeepSeek-R1-Distill-Qwen-7B-Q2_K.gguf",
        "tokenizer_path": "deepseek-ai/DeepSeek-R1-7B"
    },
    {
        "repo_id": "bartowski/DeepSeek-R1-Distill-Llama-8B-GGUF",
        "filename": "DeepSeek-R1-Distill-Llama-8B-Q2_K_L.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B"
    },
    {
        "repo_id": "bartowski/DeepSeek-R1-Distill-Llama-8B-GGUF",
        "filename": "DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B"
    },
]



quantized_model_info = [ 
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-1B-Instruct-GGUF",
        "filename": "Llama-3.2-1B-Instruct.IQ1_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-1B-Instruct-GGUF",
        "filename": "Llama-3.2-1B-Instruct.Q2_K.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-1B-Instruct-GGUF",
        "filename": "Llama-3.2-1B-Instruct.Q4_K_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-1B-Instruct-GGUF",
        "filename": "Llama-3.2-1B-Instruct.Q8_0.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-3B-Instruct-GGUF",
        "filename": "Llama-3.2-3B-Instruct.IQ1_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-3B-Instruct-GGUF",
        "filename": "Llama-3.2-3B-Instruct.Q2_K.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-3B-Instruct-GGUF",
        "filename": "Llama-3.2-3B-Instruct.Q4_K_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-3B-Instruct-GGUF",
        "filename": "Llama-3.2-3B-Instruct.Q8_0.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
    {
        "repo_id": "MaziyarPanahi/Meta-Llama-3-8B-Instruct-GGUF",
        "filename": "Meta-Llama-3-8B-Instruct.IQ1_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B",
    },
    {
        "repo_id": "MaziyarPanahi/Meta-Llama-3-8B-Instruct-GGUF",
        "filename": "Meta-Llama-3-8B-Instruct.IQ2_XS.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B",
    },
    {
        "repo_id": "MaziyarPanahi/Meta-Llama-3-8B-Instruct-GGUF",
        "filename": "Meta-Llama-3-8B-Instruct.Q4_K_S.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B",
    },
    {
        "repo_id": "MaziyarPanahi/Meta-Llama-3-8B-Instruct-GGUF",
        "filename": "Meta-Llama-3-8B-Instruct.Q8_0.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B",
    },
]

all_models = model_info + quantized_model_info

In [6]:
len(all_models)

26

<h1> Few Shot Prompt </h1>

In [7]:
def make_demo_text():
    ## https://github.com/kojima-takeshi188/zero_shot_cot/blob/main/utils.py
    x, z, y = [], [], []

    x.append("There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?")
    z.append("There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.")
    y.append("6")
                       
    x.append("Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?")
    z.append("Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9.")
    y.append("9")        

    x.append("There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?")
    z.append("There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29.")
    y.append("29")        
    demo_text = ""
    for i in range(len(z)):
        demo_text += "Q: " + x[i] + "\nA: " + z[i] + " " + \
                        "The answer is " + " " + y[i] + ".\n\n"
    
    demo_text = demo_text + "Now answer the following question. Do not continue with any more Q&A after you've answered: \n"
    return demo_text

<h1> Prompting Models </h1>

In [8]:
def load_prompts(file_path):
    """
    Loads the list of prompt strings from a JSON file.
    
    Args:
        file_path (str): Path to the JSON file containing the prompt strings.
    
    Returns:
        list: The list of prompt strings.
    """
    with open(file_path, 'r') as f:
        prompts = json.load(f)
    return prompts


prompt_dataset = load_prompts("Processed_Prompts.json")

In [9]:
len(prompt_dataset)

1305

In [ ]:
max_tokens = 1000
number_of_prompts_to_sample = 100
keep_previous_run = True
# model = model_info[0]

for model in all_models:
    
    model_name_original = model['filename']
    model_name_sanitized = model_name_original.replace("/", "_")
    model_name_sanitized = model_name_sanitized.replace(".", "_")
    
    
    # Create output directory if it doesn't exist
    os.makedirs("model_outputs", exist_ok=True)
    
    # Create the output path using the sanitized model name
    output_file_path = f"model_outputs/{model_name_sanitized}.txt"
    if not keep_previous_run:
        if os.path.exists(output_file_path):
            os.remove(output_file_path)
    
    # Initialize the CPU model once
    cpu_model = CPUModel(model)
    
    print(f"\nTesting model: {model_name_original} -> Output file: {output_file_path}")
    
    # Open in append mode ("a") so you can keep adding results in the same file
    with open(output_file_path, "a", encoding="utf-8") as outfile:
        # Wrap the prompt iteration in a tqdm progress bar
        for prompt_index in tqdm(range(number_of_prompts_to_sample), desc="Processing prompts"):
            try:
                # Prepare your prompt data
                demo_text = make_demo_text()
                prompt_index += 30 # NOTE: THIS SHIFTS THE PROMPTS :
                full_prompt_data = prompt_dataset[prompt_index]
                question = full_prompt_data['prompt']
                dataset_answer = full_prompt_data['answer']
                prompt_to_send = demo_text + question
    
                # Build the prompt dictionary
                prompt_dict = {
                    "prompt": prompt_to_send,
                    "auth": "access token",
                    "max_tokens": max_tokens, 
                    "stop": ["\nQ:", "\n\nQ:", "Q:", ".\n\nQ:"]
                }
    
                # print(f"\nRunning with max tokens: {max_tokens} for prompt #{prompt_index}")
                
                # Evaluate the prompt
                result = cpu_model.evaluate(prompt_dict)
                # print(result)  # For console visibility
    
                # Gather results
                results_dict = {
                    "model": model_name_original,    # Keep original name here if you want
                    "prompt_index": prompt_index,
                    "prompt": prompt_to_send,
                    "model_output": result['model_output_str'],
                    "latency": result['latency'],
                    "throughput": result['throughput'],
                    "max_ram": result['max_ram'],
                    "tokens_generated": result['tokens_generated'],
                    "answer": dataset_answer
                }
    
                # Write the results to file (one line per prompt)
                outfile.write(f"{results_dict}\n")
    
            except Exception as e:
                # Error check
                print(f"Error occurred for prompt_index={prompt_index}: {e}")
            
            finally:
                try:
                    # Delete large or unneeded variables to minimize RAM usage
                    del demo_text
                    del full_prompt_data
                    del question
                    del dataset_answer
                    del prompt_to_send
                    del prompt_dict
                    del result
                    del results_dict
                except:
                    print("Error in deleting variables")
            # time.sleep(2)
    
    print("\nAll prompts processed for model:", model_name_sanitized)
    del cpu_model
    time.sleep(10)
    


<h1> Eval Code </h1>

In [11]:
# https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
def read_model_outputs(model_name):
    """
    Reads the output file for the given model and returns a list of dictionaries,
    each representing the results for one prompt.

    Args:
        model_name (str): The original model name (e.g., "some/model").

    Returns:
        list: A list of dictionaries read from the file.
    """
    # Sanitize the model name (replace "/" with "_") to match the file naming
    model_name_sanitized = model_name.replace("/", "_")
    model_name_sanitized = model_name_sanitized.replace(".", "_")
    file_path = f"model_outputs/{model_name_sanitized}.txt"
    
    results = []
    
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"No output file found for model '{model_name}'.")
        return results
    
    # Open and read the file line by line
    with open(file_path, "r", encoding="utf-8") as infile:
        for line in infile:
            line = line.strip()
            if line:
                try:
                    # Convert the string representation of the dictionary into a dict
                    result_dict = ast.literal_eval(line)
                    results.append(result_dict)
                except Exception as e:
                    print(f"Error parsing line: {line}\nError: {e}")
                    
    return results

In [12]:
def eval_answer(model_output, answer_key):
    """
    Evaluates the model output by extracting the first occurrence of text following 
    "The answer is" and comparing it to answer_key. If answer_key is numeric (or can be converted 
    to a float), then the function extracts a numeric value, accepting an optional leading '$' 
    and commas. Otherwise, it extracts a string.

    Returns:
        1 if the extracted answer matches answer_key, 0 otherwise.
    """
    # Determine whether answer_key is numeric.
    try:
        answer_key_numeric = float(answer_key)
        answer_key_is_numeric = True
    except (ValueError, TypeError):
        answer_key_is_numeric = False

    if answer_key_is_numeric:
        # Use a regex that captures a number, allowing an optional '$' and commas.
        # This regex expects: "The answer is" followed by optional whitespace, an optional '$',
        # then a number that may include commas and an optional decimal.
        match = re.search(r"The answer is\s+\$?([-+]?\d[\d,]*\.?\d*)", model_output)
        if not match:
            print("No numeric answer found in model output.")
            return 0
        extracted_str = match.group(1)
        # Remove commas from the extracted number.
        extracted_str = extracted_str.replace(',', '')
        try:
            extracted_num = float(extracted_str)
        except ValueError:
            print("Failed to convert extracted numeric string to float.")
            return 0
        # Print what is being compared
        print(f"Comparing extracted answer: {extracted_num} with answer key: {answer_key_numeric}")
        return 1 if extracted_num == answer_key_numeric else 0
    else:
        # If answer_key is not numeric, extract a string token.
        # This regex captures everything until a punctuation (period, exclamation, newline) or end-of-string.
        match = re.search(r"The answer is\s+(.+?)(?:[.!\n]|$)", model_output)
        if not match:
            print("No answer found in model output.")
            return 0
        extracted_str = match.group(1).strip()
        # Optionally, remove trailing punctuation if present.
        extracted_str = extracted_str.rstrip('.,')
        # Print what is being compared
        print(f"Comparing extracted answer: '{extracted_str}' with answer key: '{str(answer_key)}'")
        return 1 if extracted_str == str(answer_key) else 0

# Example usage:
# example1_model = (" The answer is  10. Janet has 16 eggs per day, so 16 - 3 = 13 eggs left. "
#                   "13 eggs * 4 eggs per muffin = 52 muffins per day. 52 muffins * $2 per muffin = 104 dollars per day. "
#                   "The answer is  104.")
# example1_key = 18.0  # Expected to print the comparison and return 0

# example2_model = "2 bolts of blue fiber, and 2.5 bolts of white fiber. 2 + 2.5 = 4.5. The answer is  4.5."
# example2_key = 3.0   # Expected to print the comparison and return 0

# print("Result for example 1:", eval_answer(example1_model, example1_key))
# print("Result for example 2:", eval_answer(example2_model, example2_key))


In [15]:
for model in all_models:
    print('Evaluating:', model['filename'])
    data_out = read_model_outputs(model['filename'])
    eval_result = 0
    count = 0
    for i in range(len(data_out)):
        eval_result += eval_answer(data_out[i]['model_output'], data_out[i]['answer'])
        count += 1
    print(f"Accuracy: {eval_result/count}\n")

Evaluating: Llama-3.2-1B-Instruct.Q4_K_M.gguf
Comparing extracted answer: 3.0 with answer key: 18.0
Comparing extracted answer: 2.0 with answer key: 3.0
No numeric answer found in model output.
Comparing extracted answer: 540.0 with answer key: 540.0
Comparing extracted answer: 40.0 with answer key: 20.0
Comparing extracted answer: 50.0 with answer key: 64.0
Comparing extracted answer: 260.0 with answer key: 260.0
Comparing extracted answer: 60.0 with answer key: 160.0
No numeric answer found in model output.
Comparing extracted answer: 412.0 with answer key: 460.0
Comparing extracted answer: 120.0 with answer key: 366.0
Comparing extracted answer: 204.0 with answer key: 694.0
No numeric answer found in model output.
Comparing extracted answer: 6.65 with answer key: 18.0
Comparing extracted answer: 15.0 with answer key: 60.0
Comparing extracted answer: 221.0 with answer key: 125.0
Comparing extracted answer: 30.0 with answer key: 230.0
No numeric answer found in model output.
Comparing

In [24]:
demo_text = make_demo_text()
full_prompt_data = prompt_dataset[0]
question = full_prompt_data['prompt']
dataset_answer = full_prompt_data['answer']
prompt_to_send = demo_text + question

# Build the prompt dictionary
prompt_dict = {
    "prompt": prompt_to_send,
    "auth": "access token",
    "max_tokens": 500, 
    "stop": ["\nQ:", "\n\nQ:", "Q:", ".\n\nQ:"]
}
print(prompt_dict['prompt'])

Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is  6.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is  9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is  29.

Now answer the following question. Do not continue with any more Q&A after you've answered: 
Q: Janet’s du

In [27]:
cpu_model = CPUModel(model_info[-6])
cpu_model.evaluate(prompt_dict)

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_b

{'latency': 5.1058080196380615,
 'throughput': 8.22592620765582,
 'max_ram': 1.9077911376953125,
 'tokens_generated': 42,
 'model_output_str': '@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@'}

In [22]:
prompt_dataset[0:3]

[{'prompt': "Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\nA: ",
  'answer': 18.0},
 {'prompt': 'Q: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?\nA: ',
  'answer': 3.0},
 {'prompt': 'Q: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?\nA: ',
  'answer': 70000.0}]

# Running with Batches

In [10]:
batch_models = [
    {
        "repo_id": "Qwen/Qwen2-0.5B-Instruct-GGUF",
        "filename": "qwen2-0_5b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "Qwen/Qwen2-1.5B-Instruct-GGUF",
        "filename": "qwen2-1_5b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "Qwen/Qwen2-7B-Instruct-GGUF",
        "filename": "qwen2-7b-instruct-q4_k_m.gguf",
        "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
    },
    {
        "repo_id": "MaziyarPanahi/Llama-3.2-3B-Instruct-GGUF",
        "filename": "Llama-3.2-3B-Instruct.Q4_K_M.gguf",
        "tokenizer_path": "meta-llama/Meta-Llama-3-8B", 
    },
]

In [ ]:
import torch
import time
import psutil
import gc
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Model and Tokenizer
model_name = "unsloth/Llama-3.2-3B"  # changed to appropiate model as needed
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32, device_map="cpu")
model.eval()

# Extract prompts
NUM_PROMPTS = 10
prompts = [item["prompt"] for item in prompt_dataset[:NUM_PROMPTS]]

# Define batch sizes to test
batch_sizes = [1, 4, 10]  # Adjust based on your CPU capacity

# Store results
results = []

# Define the directory where you want to save the file
output_dir = "batch_size_benchmark_results"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Run benchmarking for different batch sizes
for batch_size in batch_sizes:
    print(f"\nRunning inference with batch size: {batch_size}")
    
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i: i + batch_size]

        # Tokenize
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs["input_ids"]

        # Measure RAM and CPU usage before inference
        gc.collect()
        process = psutil.Process()
        mem_before = process.memory_info().rss / (1024 ** 3)  # GB
        cpu_before = psutil.cpu_percent(interval=0.1)  # Measure CPU utilization

        # Start timing
        start_time = time.time()

        # Run inference
        with torch.no_grad():
            outputs = model.generate(input_ids, max_length=1000)

        # Measure time taken
        latency = time.time() - start_time

        # Measure RAM and CPU usage after inference
        mem_after = process.memory_info().rss / (1024 ** 3)  # GB
        cpu_after = psutil.cpu_percent(interval=0.1)  # Measure CPU utilization after inference
        max_ram = mem_after  # RAM usage after inference
        cpu_usage = cpu_after - cpu_before  # CPU usage during the batch

        # Decode outputs
        responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        # Compute throughput (tokens per second)
        total_tokens = sum(len(tokenizer.encode(resp)) for resp in responses)  # Count tokens per response
        throughput = total_tokens / latency if latency > 0 else 0

        # Store results for each prompt
        for prompt_index, response in enumerate(responses):
            result = {
                "batch_size": batch_size,
                "prompt_index": i + prompt_index,  # keep the correct index
                "prompt": batch_prompts[prompt_index],
                "model_output_str": response,
                "latency": latency,
                "throughput": throughput,
                "max_ram": max_ram,
                "cpu_usage": cpu_usage,
                "tokens_generated": len(tokenizer.encode(response))  # tokens generated per prompt
            }
            results.append(result)

        # Print results for the batch
        print(f"Batch {i // batch_size + 1}: Latency: {latency:.4f}s, RAM: {max_ram:.2f}GB, CPU Usage: {cpu_usage:.2f}%, Throughput: {throughput:.2f} tokens/s")

# Convert results to DataFrame
df = pd.DataFrame(results)

# Save to CSV
csv_filename = f"{output_dir}/batch_size_benchmark_{model_name.split('/')[-1]}.csv"
df.to_csv(csv_filename, index=False)

print(f"\nBenchmarking complete. Results saved to '{csv_filename}'.")



Running inference with batch size: 1
Batch 1: Latency: 19.3911s, RAM: 12.24GB, CPU Usage: 77.70%, Throughput: 6.19 tokens/s
Batch 2: Latency: 169.9687s, RAM: 12.71GB, CPU Usage: 75.90%, Throughput: 5.88 tokens/s
Batch 3: Latency: 10.5361s, RAM: 12.71GB, CPU Usage: 76.10%, Throughput: 11.29 tokens/s
Batch 4: Latency: 3.7975s, RAM: 12.69GB, CPU Usage: 80.50%, Throughput: 16.33 tokens/s
Batch 5: Latency: 3.9931s, RAM: 12.67GB, CPU Usage: 76.80%, Throughput: 33.56 tokens/s
Batch 6: Latency: 417.8116s, RAM: 6.87GB, CPU Usage: 83.20%, Throughput: 2.39 tokens/s
Batch 7: Latency: 786.9592s, RAM: 11.90GB, CPU Usage: 83.00%, Throughput: 1.27 tokens/s
Batch 8: Latency: 788.3817s, RAM: 12.69GB, CPU Usage: 78.60%, Throughput: 1.27 tokens/s
Batch 9: Latency: 747.0839s, RAM: 12.43GB, CPU Usage: 86.50%, Throughput: 1.34 tokens/s
Batch 10: Latency: 660.0135s, RAM: 12.35GB, CPU Usage: 75.10%, Throughput: 1.52 tokens/s

Running inference with batch size: 4


In [21]:
import os
import time
import torch
import psutil
from tqdm import tqdm

max_tokens = 1000
number_of_prompts_to_sample = 100
keep_previous_run = False
batch_size = 10  # Configurable batch size

for model in batch_models:
    
    model_name_original = model['filename'] + "_BATCH10"
    model_name_sanitized = model_name_original.replace("/", "_").replace(".", "_")
    
    os.makedirs("model_outputs", exist_ok=True)
    output_file_path = f"model_outputs/{model_name_sanitized}.txt"

    if not keep_previous_run and os.path.exists(output_file_path):
        os.remove(output_file_path)

    cpu_model = BatchCPUModel(model)
    
    print(f"\nTesting model: {model_name_original} -> Output file: {output_file_path}")

    with open(output_file_path, "a", encoding="utf-8") as outfile:
        for batch_start in tqdm(range(0, number_of_prompts_to_sample, batch_size), desc="Processing batches"):
            batch_end = min(batch_start + batch_size, number_of_prompts_to_sample)
            batch_prompts = []
            batch_indices = []

            for prompt_index in range(batch_start, batch_end):
                try:
                    demo_text = make_demo_text()
                    full_prompt_data = prompt_dataset[prompt_index]
                    question = full_prompt_data['prompt']
                    dataset_answer = full_prompt_data['answer']
                    prompt_to_send = demo_text + question
                    prompt_dict = {
                        "prompt": prompt_to_send,
                        "auth": "access token",
                        "max_tokens": max_tokens, 
                        "stop": ["\nQ:", "\n\nQ:", "Q:", ".\n\nQ:"]
                    }
                    batch_prompts.append(prompt_dict)
                    batch_indices.append(prompt_index)

                except Exception as e:
                    print(f"Error preparing prompt_index={prompt_index}: {e}")

            if not batch_prompts:
                continue  # Skip if batch is empty due to errors

            try:
                batch_results = cpu_model.evaluate(batch_prompts)

                for idx, result in enumerate(batch_results):
                    prompt_index = batch_indices[idx]
                    results_dict = {
                        "model": model_name_original,
                        "prompt_index": prompt_index,
                        "prompt": batch_prompts[idx]['prompt'],
                        "model_output": result['model_output_str'],
                        "latency": result['latency'],
                        "throughput": result['throughput'],
                        "max_ram": result['max_ram'],
                        "tokens_generated": result['tokens_generated'],
                        "answer": prompt_dataset[prompt_index]['answer']
                    }
                    outfile.write(f"{results_dict}\n")

            except Exception as e:
                print(f"Error processing batch starting at index {batch_start}: {e}")

            finally:
                time.sleep(2)  # Optional: Reduce CPU load

    print("\nAll prompts processed for model:", model_name_sanitized)
    del cpu_model
    time.sleep(10)


llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf


Testing model: qwen2-0_5b-instruct-q4_k_m.gguf_BATCH10 -> Output file: model_outputs/qwen2-0_5b-instruct-q4_k_m_gguf_BATCH10.txt


Processing batches: 100%|███████████████████████| 10/10 [02:19<00:00, 13.94s/it]



All prompts processed for model: qwen2-0_5b-instruct-q4_k_m_gguf_BATCH10


llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf

...


All prompts processed for model: qwen2-7b-instruct-q4_k_m_gguf_BATCH10


llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_b


Testing model: Llama-3.2-3B-Instruct.Q4_K_M.gguf_BATCH10 -> Output file: model_outputs/Llama-3_2-3B-Instruct_Q4_K_M_gguf_BATCH10.txt


Processing batches: 100%|███████████████████████| 10/10 [05:15<00:00, 31.59s/it]



All prompts processed for model: Llama-3_2-3B-Instruct_Q4_K_M_gguf_BATCH10


# Comparing PyTorch to Llama.cpp

In [15]:
comparison_model = {
    "repo_id": "Qwen/Qwen2-0.5B-Instruct-GGUF",
    "filename": "qwen2-0_5b-instruct-fp16.gguf",
    "tokenizer_path": "Qwen/Qwen2.5-1.5B-Instruct"
}

In [16]:
import torch
import time
import psutil
import gc
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Model and Tokenizer
model_name = "Qwen/Qwen2-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

# Load the model in bfloat16 for reduced memory usage (better for CPUs)
pytorch_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # bfloat16 is better for CPU than float16
    device_map="cpu"
)
pytorch_model.eval()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [18]:
# Number of prompts to process
NUM_PROMPTS = 10
prompts = [item["prompt"] for item in prompt_dataset[:NUM_PROMPTS]]

# Store results
results = []

# Output directory
output_dir = "pytorch_v_llamacpp"
os.makedirs(output_dir, exist_ok=True)

# Run inference for each prompt
for i, prompt in enumerate(prompts):
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"]

    # Measure RAM and CPU usage before inference
    gc.collect()
    process = psutil.Process()
    mem_before = process.memory_info().rss / (1024 ** 3)  # Convert to GB
    cpu_before = psutil.cpu_percent(interval=0.1)

    # Start timing
    start_time = time.time()

    # Run inference
    with torch.no_grad():
        outputs = pytorch_model.generate(input_ids, max_length=1000)

    # Measure time taken
    latency = time.time() - start_time

    # Measure RAM and CPU usage after inference
    mem_after = process.memory_info().rss / (1024 ** 3)  # Convert to GB
    cpu_after = psutil.cpu_percent(interval=0.1)
    max_ram = mem_after  # RAM usage after inference
    cpu_usage = cpu_after - cpu_before  # CPU usage during inference

    # Decode output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Compute throughput (tokens per second)
    tokens_generated = len(tokenizer.encode(response))
    throughput = tokens_generated / latency if latency > 0 else 0

    # Store results
    results.append({
        "prompt_index": i,
        "prompt": prompt,
        "model_output_str": response,
        "latency": latency,
        "throughput": throughput,
        "max_ram": max_ram,
        "cpu_usage": cpu_usage,
        "tokens_generated": tokens_generated
    })

    # Print results for this prompt
    print(f"{i}: Latency: {latency:.4f}s, RAM: {max_ram:.2f}GB, CPU Usage: {cpu_usage:.2f}%, Throughput: {throughput:.2f} tokens/s")

# Convert results to DataFrame and save to CSV
df = pd.DataFrame(results)
csv_filename = f"{output_dir}/pytorch_{model_name.split('/')[-1]}.csv"
df.to_csv(csv_filename, index=False)

print(f"\nBenchmarking complete. Results saved to '{csv_filename}'.")


0: Latency: 12.9553s, RAM: 2.53GB, CPU Usage: 87.70%, Throughput: 10.34 tokens/s
1: Latency: 45.0157s, RAM: 2.53GB, CPU Usage: 78.00%, Throughput: 8.75 tokens/s
2: Latency: 21.0511s, RAM: 2.53GB, CPU Usage: 89.50%, Throughput: 9.55 tokens/s
3: Latency: 118.7269s, RAM: 2.54GB, CPU Usage: 89.70%, Throughput: 8.42 tokens/s
4: Latency: 116.2354s, RAM: 2.54GB, CPU Usage: 90.60%, Throughput: 8.54 tokens/s
5: Latency: 119.9791s, RAM: 2.55GB, CPU Usage: 86.60%, Throughput: 8.33 tokens/s
6: Latency: 122.6044s, RAM: 2.55GB, CPU Usage: 63.00%, Throughput: 8.16 tokens/s
7: Latency: 18.8680s, RAM: 2.55GB, CPU Usage: 74.10%, Throughput: 9.80 tokens/s
8: Latency: 124.6146s, RAM: 2.57GB, CPU Usage: 72.70%, Throughput: 8.02 tokens/s
9: Latency: 28.8057s, RAM: 2.57GB, CPU Usage: 78.80%, Throughput: 9.20 tokens/s

Benchmarking complete. Results saved to 'pytorch_v_llamacpp/pytorch_Qwen2-0.5B-Instruct.csv'.


In [29]:
max_tokens = 1000
number_of_prompts_to_sample = 10
results = []

# Initialize the CPU model once
cpu_model = CPUModel(comparison_model)

# Wrap the prompt iteration in a tqdm progress bar
for prompt_index in tqdm(range(number_of_prompts_to_sample), desc="Processing prompts"):
    try:
        # Prepare your prompt data
        demo_text = make_demo_text()
        full_prompt_data = prompt_dataset[prompt_index]
        question = full_prompt_data['prompt']
        dataset_answer = full_prompt_data['answer']
        prompt_to_send = demo_text + question

        # Build the prompt dictionary
        prompt_dict = {
            "prompt": prompt_to_send,
            "auth": "access token",
            "max_tokens": max_tokens, 
            "stop": ["\nQ:", "\n\nQ:", "Q:", ".\n\nQ:"]
        }

        # print(f"\nRunning with max tokens: {max_tokens} for prompt #{prompt_index}")
        gc.collect()
        process = psutil.Process()
        cpu_before = psutil.cpu_percent(interval=0.1)
        
        # Evaluate the prompt
        result = cpu_model.evaluate(prompt_dict)
        # print(result)  # For console visibility

        cpu_after = psutil.cpu_percent(interval=0.1)
        cpu_usage = cpu_after

        # Gather results
        results_dict = {
            "model": model_name_original,    # Keep original name here if you want
            "prompt_index": prompt_index,
            "prompt": prompt_to_send,
            "model_output": result['model_output_str'],
            "latency": result['latency'],
            "throughput": result['throughput'],
            "max_ram": result['max_ram'],
            "tokens_generated": result['tokens_generated'],
            "answer": dataset_answer,
            "cpu_usage": cpu_usage
        }

        # Write the results to file (one line per prompt)
        results.append(results_dict)

        print(f"{prompt_index}: Latency: {results_dict['latency']:.4f}s, RAM: {results_dict['max_ram']:.2f}GB, CPU Usage: {cpu_usage:.2f}%, Throughput: {results_dict['throughput']:.2f} tokens/s")

    except Exception as e:
        # Error check
        print(f"Error occurred for prompt_index={prompt_index}: {e}")
    
    finally:
        try:
            # Delete large or unneeded variables to minimize RAM usage
            del demo_text
            del full_prompt_data
            del question
            del dataset_answer
            del prompt_to_send
            del prompt_dict
            del result
            del results_dict
        except:
            print("Error in deleting variables")


    # time.sleep(2)

print("\nAll prompts processed for model")
del cpu_model

df = pd.DataFrame(results)
csv_filename = f"{output_dir}/llamacpp_Qwen2-0.5B-Instruct-GGUF.csv"
df.to_csv(csv_filename, index=False)

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf

0: Latency: 2.0865s, RAM: 3.98GB, CPU Usage: 6.80%, Throughput: 78.12 tokens/s


Processing prompts:  20%|████▊                   | 2/10 [00:04<00:19,  2.41s/it]

1: Latency: 2.1034s, RAM: 3.98GB, CPU Usage: 0.80%, Throughput: 95.56 tokens/s


Processing prompts:  30%|███████▏                | 3/10 [00:07<00:16,  2.32s/it]

...

7: Latency: 1.7581s, RAM: 3.98GB, CPU Usage: 0.90%, Throughput: 89.87 tokens/s


Processing prompts:  90%|█████████████████████▌  | 9/10 [00:18<00:01,  1.96s/it]

8: Latency: 1.4455s, RAM: 3.98GB, CPU Usage: 5.10%, Throughput: 85.09 tokens/s


Processing prompts: 100%|███████████████████████| 10/10 [00:20<00:00,  2.09s/it]

9: Latency: 1.8641s, RAM: 3.98GB, CPU Usage: 3.50%, Throughput: 89.59 tokens/s

All prompts processed for model
